# Problem Statement


In the fintech sector, making informed investment decisions involves extensive analysis of multiple data points such as stock performance, market sentiment, financial reports, and macroeconomic trends. This analysis is time-consuming, complex, and requires significant expertise, often leading to missed opportunities or suboptimal investment outcomes. Investors and analysts need a more efficient, accurate, and automated way to navigate this sea of data to make well-informed decisions regarding stock investments—whether to buy, hold, or sell. Fin-Agent solves this problem by automating the process.


**Process Flow**

Input Query: Investors ask the agent whether to sell or hold a specific stock.

Data Collection: The agent gathers relevant financial metrics, news articles, insider transactions, and market trends.

Data Processing: Financial ratios are computed, insider actions are evaluated, and sentiment analysis is conducted on the news.

Recommendation: Based on the data, the agent provides a recommendation (e.g., "Consider selling the stock").

Financial Analysis: The analysis includes key financial metrics such as

Debt-to-Equity Ratio

Current Ratio

Price-to-Earnings (P/E) Ratio

Price-to-Book (P/B) Ratio These are analyzed to understand the company's financial health and its implications on stock performance.

Explanation: By calculating common financial ratios, the agent can assess whether the company is financially sound, which impacts the final decision on stock recommendation.

Insider Transactions: Insider activities, such as the buying or selling of shares by executives, provide valuable insights. In this case, significant disposals of shares by a key executive are flagged as a potential red flag.

Explanation: Monitoring insider transactions helps investors gauge how those with the most knowledge of the company’s performance feel about its future.

Market Sentiment and News Insights: The agent scrapes news sources to capture the latest market trends and investor sentiment, providing a holistic view of the stock's standing within the broader economy.

Explanation: Market sentiment, driven by external factors like interest rate cuts and sector performance, is a vital component for forecasting stock movement.

# Installations

In [ ]:
!pip install langchain_openai
!pip install --upgrade langchain_community
!pip install eventregistry

In [2]:
import os
import json
import requests
import pandas as pd

from eventregistry import *

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.tools import tool
from langchain_openai import AzureChatOpenAI



### Overview

At the core of the system is **Fin-Agent**, a higher-level agent responsible for analyzing stocks and making buy, sell, or hold recommendations. Fin-Agent uses the following tools:

- **SQL Agent**: Converts natural language queries into SQL commands to extract stock details, fundamentals, ratios, and technicals from a database.
  
- **News Agent**: Generates relevant keywords to fetch company and industry news. It then summarizes the articles for Fin-Agent to evaluate.

- **Insider Transactions Tool**: Provides data on insider trades, such as executive buy/sell actions, which can influence stock decisions.

- **Current Stock Data Tool**: Retrieves the latest stock prices, trends, and performance metrics.

### Process
Fin-Agent assesses individual or organization-owned stocks by using these tools to decide whether to hold or sell. It can also analyze any stock to provide a buy, sell, or hold recommendation.


## Note: We only have data till 2022-09 in the database. 

## Creating SQL agent

In [ ]:
# Load the configuration from the JSON file
with open('config_4o.json', 'r') as config_file:
    config = json.load(config_file)

# Extract the OpenAI key and endpoint from the configuration
openai_key = config["AZURE_OPENAI_KEY"]
azure_endpoint = config["AZURE_OPENAI_ENDPOINT"]
model_name = config["CHATGPT_MODEL"]
api_version = config["AZURE_OPENAI_APIVERSION"]

In [3]:
# Define the location of the SQLite database
db_loc = 'stock_db.db'

# Create a SQLDatabase instance from the SQLite database URI
db = SQLDatabase.from_uri(f"sqlite:///{db_loc}")

# Retrieve the schema information of the database tables
database_schema = db.get_table_info()

In [ ]:
# Initialize the AzureChatOpenAI model with the extracted configuration
gpt4o_azure = AzureChatOpenAI(
    model_name=model_name,
    api_key=openai_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
    temperature=0
)

In [ ]:
# Define the system message for the agent, including instructions and available tables
system_message = f"""current date is 2022-09-30. You are a SQLite expert agent designed to interact with a SQLite database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 100 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database..
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.
Before you execute the query, tell us why you are executing it and what you expect to find briefly.
Only use the following tables:
{database_schema}
"""

# Create a full prompt template for the agent using the system message and placeholders
full_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", '{input}'),
        MessagesPlaceholder("agent_scratchpad")
    ]
)

In [ ]:
# Create the SQL agent using the AzureChatOpenAI model, database, and prompt template
sqlite_agent = create_sql_agent(
    llm=gpt4o_azure,
    db=db,
    prompt=full_prompt,
    agent_type="openai-tools",
    agent_executor_kwargs={'handle_parsing_errors': True},
    max_iterations=5,
    verbose=True
)

#### Let's convert the sql agent into a tool that our fin agent can use.

In [4]:
@tool
def sql_tool(user_input):
    """
    Executes a SQL query using the sqlite_agent and returns the result.
    
    Args:
        user_input (str): The SQL query to be executed.
        
    Returns:
        str: The result of the SQL query execution. If an error occurs, the exception is returned as a string.
    """
    try:
        # Invoke the sqlite_agent with the user input (SQL query)
        response = sqlite_agent.invoke(user_input)

        # Extract the output from the response
        prediction = response['output']

    except Exception as e:
        # If an exception occurs, capture the exception message
        prediction = e
        
    # Return the result or the exception message
    return prediction

#### Let's test the SQL tool that we have created

In [5]:
# Test the sql_tool function with a sample user input
user_input = "What are the fundamentals of apple - ticker AAPL for today?"
answer = sql_tool.invoke(user_input)
print(answer)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT "date", "ttm_net_eps", "pe_ratio", "ttm_sales_per_share", "book_value_per_share", "ttm_fcf_per_share", "current_assets", "current_ratio", "current_assets_inventory", "quick_ratio", "total_liabilities", "debt_to_equity_ratio", "return_on_equity", "return_on_assets", "return_on_investment", "tangible_equity", "return_on_tangible_equity", "price", "current_liabilities", "shareholders_equity", "ttm_net_income", "cash_on_hand", "total_share_holder_equity", "revenue", "gross_profit", "operating_income", "ebitda", "net_income", "eps_earnings_per_share_diluted", "shares_outstanding", "ps_ratio", "pb_ratio", "pfcf_ratio", "total_assets" FROM fundamentals WHERE "ticker" = \'AAPL\' AND "date" = \'2022-09-30\''}`


```sql
SELECT "date", "ttm_net_eps", "pe_ratio", "ttm_sales_per_share", "book_value_per_share", "ttm_fcf_per_share", "current_assets", "current_ratio", "current_assets_inventory", "qui

In [25]:
## 2
user_input = "Provide me the financial information of microsoft like revenue profit and EBIDTA for today"
answer = sql_tool.invoke(user_input)
print(answer)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query` with `{'query': 'SELECT "ticker", "date", "revenue", "gross_profit", "operating_income", "ebitda", "net_income", "eps_earnings_per_share_diluted", "shares_outstanding" FROM "revenue_profit" WHERE "ticker" = \'MSFT\' AND "date" = \'2022-09-30\' LIMIT 100;'}`


[('MSFT', '2022-09-30', 50122.0, 34670.0, 21518.0, 24308.0, 17556.0, 2.35, 7485.0)]
Invoking: `sql_db_query` with `{'query': 'SELECT "ticker", "date", "total_assets", "cash_on_hand", "long_term_debt_excl_capitalized_leases", "total_liabilities", "total_share_holder_equity" FROM "assets_liabilities" WHERE "ticker" = \'MSFT\' AND "date" = \'2022-09-30\' LIMIT 100;'}`


[('MSFT', '2022-09-30', 359784.0, 107262.0, 45374.0, 186218.0, 173566.0)]
Invoking: `sql_db_query` with `{'query': 'SELECT "ticker", "date", "ttm_net_eps", "pe_ratio", "ttm_sales_per_share", "book_value_per_share", "ttm_fcf_per_share", "current_assets", "current_ratio", "quick_ratio", "total_liabil

In [3]:
## 3
user_input = "What are all the company stock values present in the database. provide me the unique companies (ticker) in the price table?"
answer = sql_tool.invoke(user_input)
print(answer)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT DISTINCT "ticker" FROM "price";'}`


```sql
SELECT DISTINCT "ticker" FROM "price";
```
Invoking: `sql_db_query` with `{'query': 'SELECT DISTINCT "ticker" FROM "price";'}`


[('AAPL',), ('ABBV',), ('ABT',), ('ACN',), ('ADBE',), ('AMZN',), ('ASML',), ('AVGO',), ('AZN',), ('BABA',), ('BAC',), ('BMY',), ('BRK.A',), ('CMCSA',), ('COP',), ('COST',), ('CSCO',), ('CVX',), ('DHR',), ('DIS',), ('GOOGL',), ('HD',), ('JNJ',), ('JPM',), ('KO',), ('LIN',), ('LLY',), ('MA',), ('MCD',), ('META',), ('MRK',), ('MSFT',), ('NEE',), ('NKE',), ('NVDA',), ('NVS',), ('ORCL',), ('PEP',), ('PFE',), ('PG',), ('PM',), ('RTX',), ('SCHW',), ('SHEL',), ('TMO',), ('TMUS',), ('TSLA',), ('TTE',), ('TXN',), ('UNH',), ('UPS',), ('V',), ('VZ',), ('WFC',), ('WMT',), ('XOM',)]The unique companies (tickers) present in the price table are:

- AAPL
- ABBV
- ABT
- ACN
- ADBE
- AMZN
- ASML
- AVGO
- AZN
- BABA
- BAC
- BMY
- BRK.

In [61]:
## 4
user_input = "Which is the industry pe value for apple?"
answer = sql_tool.invoke(user_input)
print(answer)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT "pe_ratio" FROM industry_pe_ratios WHERE "industry" = (SELECT "industry" FROM industry_ticker_mapping WHERE "ticker" = \'AAPL\')'}`


```sql
SELECT "pe_ratio" FROM industry_pe_ratios WHERE "industry" = (SELECT "industry" FROM industry_ticker_mapping WHERE "ticker" = 'AAPL')
```
Invoking: `sql_db_query` with `{'query': 'SELECT "pe_ratio" FROM industry_pe_ratios WHERE "industry" = (SELECT "industry" FROM industry_ticker_mapping WHERE "ticker" = \'AAPL\')'}`


[(32.28,)]The industry PE ratio for Apple (AAPL) is 32.28.

> Finished chain.
The industry PE ratio for Apple (AAPL) is 32.28.


## Creating News Researcher Agent

At first, we tried to create seperate tools for company and industry level news, however they overloadede the top-level agent [Fin-Agent] with a lot of information and combined with other tools' output they exceeded the context length of the top level agent. To overcome this, we created a seperate agent for these tools that captures news from them and then summarises it and sends it up to the top-level agent. We found this a nice, practical case where we need multiple agents in a system instead of a lot of tools with a single agent.

Let's start by creating tools required by this agent

Get free api key from https://newsapi.ai/

### Create tools

In [6]:
# Open the 'financial_apis.json' file and load its content into the 'fin_config' variable
with open('financial_apis.json', 'r') as fin_config:
    fin_config = json.load(fin_config)

# Extract the 'apha_api_key' from the loaded configuration and assign it to a variable
apha_api_key = fin_config['apha_api_key']

# Extract the 'news_client_id' from the loaded configuration and assign it to a variable
news_client_id = fin_config["news_client_id"]

In [8]:
# we will use the EventRegistry API to get the news data. 
er = EventRegistry(apiKey = news_client_id, allowUseOfArchive=False)

In [9]:
@tool
def company_news(company_name: str) -> list:
    """
    Searches for a collection of news articles [most relevant, most socially shared and keyword based news] based on the given company name and industry keywords, and returns the news context as a list of articles.

    Args:
        company_name (str): The name of the company for finding relevant news articles.
        industry_keywords (list): A list of keywords related to the industry.

    Returns:
        list: A list of dictionaries containing the news articles.
    """
    news = []
    try:
        # Get most relevant news articles based on the company name
        q_pos = QueryArticlesIter(
            conceptUri=er.getConceptUri(company_name), #filters for news related to the company
            categoryUri=er.getCategoryUri("investing"), #fitlers for news related to investing
            dataType=["news"],
            lang="eng"
        )
        for art in q_pos.execQuery(er, sortBy="cosSim", maxItems=10): #sort by options - cosSim, date, SourceImportance, socialScore
            news.append(art)

    except Exception as e:
    # If an exception occurs, capture the exception message
        news = e
    return news

@tool
def industry_news(industry_keywords: list) -> list:
    """
    Fetches the most relevant articles related to a given list of industry keywords.
    Args:
        industry_keywords (list): A list of keywords representing the industry of interest.
    Returns:
        list: A list of news articles related to the provided industry keywords.

    """
    news = []
    try:
        q_pos = QueryArticlesIter(
            conceptUri=QueryItems.OR([er.getConceptUri(i) for i in industry_keywords]),
            categoryUri=er.getCategoryUri("investing"),
            dataType=["news"],
            lang = "eng"
        )
        for art in q_pos.execQuery(er, sortBy="cosSim", maxItems=10):
            news.append(art)
    except Exception as e:
    # If an exception occurs, capture the exception message
        news = e
    return news



**parameter options for execQuery**

- **eventRegistry**: Instance of the `EventRegistry` class used to obtain the necessary data.

- **sortBy**: Order in which event articles are sorted. Options include:
        - `none`: No specific sorting
        - `id`: Internal ID
        - `date`: Published date
        - `cosSim`: Closeness to event centroid
        - `sourceImportance`: Manually curated score of source importance (high value, high importance)
        - `sourceImportanceRank`: Reverse of `sourceImportance`
        - `sourceAlexaGlobalRank`: Global rank of the news source
        - `sourceAlexaCountryRank`: Country rank of the news source
        - `socialScore`: Total shares on social media
        - `facebookShares`: Shares on Facebook only

- **sortByAsc**: Determines if the results should be sorted in ascending (`True`) or descending (`False`) order.

- **returnInfo**: Specifies what details should be included in the returned information.

- **maxItems**: Maximum number of items to be returned. Used to stop iteration sooner than results run out.

In [10]:
# Testing the news_tool
company_name = "Apple inc."
keywords = ["computers", "electronics"]

# Use the invoke method instead of __call__
response = industry_news.invoke({"industry_keywords": keywords})
response

[{'uri': '2024-10-503889940',
  'lang': 'eng',
  'isDuplicate': False,
  'date': '2024-10-03',
  'time': '09:36:11',
  'dateTime': '2024-10-03T09:36:11Z',
  'dateTimePub': '2024-10-03T09:35:38Z',
  'dataType': 'news',
  'sim': 0.9960784316062927,
  'url': 'https://www.streetinsider.com/PRNewswire/Vermiculus+selected+to+deliver+microservice-based+CSD+system+to+The+Bermuda+Stock+Exchange/23796784.html',
  'title': 'Vermiculus selected to deliver microservice-based CSD system to The Bermuda Stock Exchange',
  'body': 'STOCKHOLM, Oct. 3, 2024 /PRNewswire/ -- Vermiculus Financial Technology, a global provider of trading, clearing, and CSD technology, today announced it has been selected to deliver microservice-based clearing, settlement and depository (CSD) system to The Bermuda Stock Exchange (BSX), a wholly owned subsidiary of Miami International Holdings, Inc.\n\nThe new BSX CSD systems are based on Vermiculus\' VeriClear™, an elastically scalable real-time product solution ensuring the 

In [ ]:
response = company_news.invoke({"company_name": company_name})
response

[{'uri': '8337944028',
  'lang': 'eng',
  'isDuplicate': False,
  'date': '2024-09-26',
  'time': '10:31:01',
  'dateTime': '2024-09-26T10:31:01Z',
  'dateTimePub': '2024-09-26T10:30:16Z',
  'dataType': 'news',
  'sim': 0.9921568632125854,
  'url': 'https://www.theglobeandmail.com/investing/article-power-thirst-complicates-esg-investors-love-affair-with-tech-stocks/',
  'title': "Power thirst complicates ESG investors' love affair with tech stocks",
  'body': 'Investors who manage hundreds of billions of dollars are pressing Microsoft, Alphabet and others for more information on the power needed for artificial intelligence and advanced computing, to help decide whether the sector should stay heavily represented in sustainable funds, investors said.\n\nWhile these conversations are at an early stage, six fund industry executives in Europe and the United States said they are looking more closely at the environmental impact of the AI boom, which Goldman Sachs estimates will boost data cen

### Creating the News Agent using the two tools

In [11]:
system_message = f"""
You are a financial researcher who can look through a lot of news articles and extract very important insights for a news article.
You have access to tools that can give you news articles based on the company name and also based on the industry keywords.
your job is to give the right company name and pick the right industry keywords to get the most relevant news articles.

After fetching the news articles - go through them and extract the most important insights from the news articles and summarize them in bullet points. 
You can choose to not use a tool and reply 'I am not able to determine whether to hold it or not. Here is the information I do have: ' along with summary of all the information you have gathered if the tools are not useful to answer the question.
or If answer is not available in the tool output - do the same and reply with the summary of all the information you have with a note.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

tools = [company_news, industry_news]

news_llm = AzureChatOpenAI(
    model_name=model_name,
    api_key=openai_key,
    azure_endpoint=azure_endpoint,
    api_version="2024-02-01",
    temperature=0
)

news_agent = create_tool_calling_agent(news_llm, tools, prompt)
news_agent_executor = AgentExecutor(agent=news_agent, tools=tools, verbose=True)


#### Let's the news agent

In [231]:
news_agent_executor.invoke({"input": "apple inc."})



> Entering new AgentExecutor chain...

Invoking: `company_news` with `{'company_name': 'Apple Inc.'}`


[{'uri': '2024-09-496969714', 'lang': 'eng', 'isDuplicate': False, 'date': '2024-09-26', 'time': '10:31:12', 'dateTime': '2024-09-26T10:31:12Z', 'dateTimePub': '2024-09-26T06:04:00Z', 'dataType': 'news', 'sim': 0.9921568632125854, 'url': 'https://www.usnews.com/news/top-news/articles/2024-09-26/analysis-power-thirst-complicates-esg-investors-love-affair-with-tech-stocks', 'title': "Analysis-Power Thirst Complicates ESG Investors' Love Affair With Tech Stocks", 'body': 'NEW YORK (Reuters) - Investors who manage hundreds of billions of dollars are pressing Microsoft, Alphabet and others for more information on the power needed for artificial intelligence and advanced computing, to help decide whether the sector should stay heavily represented in sustainable funds, investors said.\n\nWhile these conversations are at an early stage, six fund industry executives in Europe and the United

{'input': 'apple inc.',
 'output': 'Here are the most important insights from the recent news articles related to Apple Inc. and the technology industry:\n\n### Apple Inc. Specific Insights:\n1. **Environmental Impact and ESG Concerns**:\n   - Investors managing hundreds of billions of dollars are pressing tech giants, including Apple, for more information on the power needed for AI and advanced computing.\n   - Concerns are rising about the environmental impact of the AI boom, which is expected to boost data center power demand by 160% by 2030.\n   - Apple, along with other tech companies, has started reporting increased greenhouse gas emissions due to the power and materials needed for data centers.\n   - Despite these concerns, none of the investors contacted are considering divesting from these companies.\n\n2. **Sustainability and Renewable Energy**:\n   - Apple has been part of the top holdings in ESG-focused funds, but there is pressure to maintain commitments to renewable energ

### Convert the agent into a tool so that our top-level agent can use it.

In [12]:
@tool
def news_tool(company_name:str) -> str:
    """
    Fetches the most relevant news articles based on the company name.
    It automatically fetched the most relevant keywords and extracts for the industry and then gets news related to those keywords.

    Args:
        company_name (str): The name of the company for finding relevant news articles.

    Returns:
        str: The most important insights extracted from the news articles.
    """
    try:
        response = news_agent_executor.invoke({"input": company_name})
        response = response["output"]
    except Exception as e:
        response = e
    return response

In [13]:
news_tool.invoke({"company_name":"Nvidia"})



> Entering new AgentExecutor chain...

Invoking: `company_news` with `{'company_name': 'Nvidia'}`


[{'uri': '2024-09-496969714', 'lang': 'eng', 'isDuplicate': False, 'date': '2024-09-26', 'time': '10:31:12', 'dateTime': '2024-09-26T10:31:12Z', 'dateTimePub': '2024-09-26T06:04:00Z', 'dataType': 'news', 'sim': 0.9921568632125854, 'url': 'https://www.usnews.com/news/top-news/articles/2024-09-26/analysis-power-thirst-complicates-esg-investors-love-affair-with-tech-stocks', 'title': "Analysis-Power Thirst Complicates ESG Investors' Love Affair With Tech Stocks", 'body': 'NEW YORK (Reuters) - Investors who manage hundreds of billions of dollars are pressing Microsoft, Alphabet and others for more information on the power needed for artificial intelligence and advanced computing, to help decide whether the sector should stay heavily represented in sustainable funds, investors said.\n\nWhile these conversations are at an early stage, six fund industry executives in Europe and the United Sta

"Here are the most important insights from the recent news articles about Nvidia and the semiconductor industry:\n\n### Nvidia-Specific Insights:\n1. **ESG Concerns and AI Power Consumption**:\n   - Investors managing hundreds of billions of dollars are pressing tech giants like Nvidia for more information on the power needed for AI and advanced computing.\n   - The AI boom is expected to boost data center power demand by 160% by 2030, raising concerns about increased greenhouse gas emissions.\n   - Nvidia, along with other tech giants, is heavily invested in by funds that prioritize environmental, social, and governance (ESG) concerns. These investments could be affected if the companies do not address these environmental concerns.\n\n2. **Environmental Impact**:\n   - Nvidia and other tech companies have started reporting increased greenhouse gas emissions due to the power and materials needed for data centers.\n   - There is a push for these companies to source renewable energy to m

## Create tools to fetch insider transactions and recent stock prices

Get free api key from https://www.alphavantage.co/support/#api-key

In [14]:
date= "2022-09-30"

@tool
def get_stock_price(ticker:str):
    """
    Fetches the stock price for a given ticker symbol using the Alpha Vantage API.

    Args:
        ticker (str): The ticker symbol of the stock.

    Returns:
        dict: A dataframe containing the latest stock data with 30min interval.
    """
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval=30min&month=2022-09&outputsize=full&apikey={apha_api_key}'
    r = requests.get(url)
    data = r.json()

    
    # Extract the time series data
    time_series = data['Time Series (30min)']

    # Convert the time series data into a DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')

    # Rename the columns
    df.columns = ['open', 'high', 'low', 'close', 'volume']

    # Convert the index to datetime format
    df.index = pd.to_datetime(df.index)

    # Filter for the latest date
    latest_date = df.index.max().date()
    filtered_df = df[df.index.date == latest_date]

    return filtered_df

@tool
def insider_transactions(ticker:str):
    """
    Fetches the insider transactions data for a given company ticker symbol using the Alpha Vantage API.

    Args:
        ticker (str): The ticker symbol of the company.

    Returns:
        dict: A dictionary containing the insider transactions data of the company.

    Raises:
        requests.exceptions.RequestException: If there is an issue with the HTTP request.
        ValueError: If the response cannot be parsed as JSON.
    """
    url = f'https://www.alphavantage.co/query?function=INSIDER_TRANSACTIONS&symbol={ticker}&apikey={apha_api_key}'
    r = requests.get(url)
    data = r.json()
    df = pd.DataFrame(data["data"])

    # Convert the 'shares' column to numeric type
    df['shares'] = pd.to_numeric(df['shares'])

    # Sort the DataFrame by the number of shares in descending order
    df_sorted = df.sort_values(by='shares', ascending=False)

    # Select the top 10 transactions
    top_10_transactions = df_sorted.head(10)


    return top_10_transactions



In [ ]:
@tool
def catchall(input: str):
    """
    A catchall function when other tools are not applicable
    """

    return "I am not able to determine whether to hold it or not. Here is the information I have: " + input

In [15]:
data = insider_transactions.invoke({"ticker": "NVDA"})
data

,transaction_date,ticker,executive,executive_title,security_type,acquisition_or_disposal,shares,share_price
3777,2016-03-16,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,3078820.0,0.0
3798,2016-03-16,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,3078820.0,0.0
3,2024-09-20,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,1568000.0,0.0
3037,2019-05-24,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,A,1251950.0,0.0
3036,2019-05-24,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,A,1251950.0,0.0
1818,2021-11-23,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,1218536.0,0.0
1820,2021-11-23,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,D,1218536.0,0.0
1653,2022-09-06,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Employee Stock Option (Right to Buy),D,1200000.0,0.0
1654,2022-09-06,NVDA,"HUANG, JEN HSUN","Director, President and CEO",Common Stock,A,1200000.0,3.4275
933,2024-04-22,NVDA,"NVIDIA, CORP",10% Owner,Common Stock,A,1050129.0,2.42


## Create Fin-Agent

In [ ]:
system_message = f"""
You are an expert stock trader equipped with several tools to analyze a company's financial health, stock performance, and market sentiment. 
Your task is to determine whether we should sell a specific stock based on comprehensive data analysis.
Before you start functioning, stop for a minute and think about all the tools you have and how you can use them to make an informed decision. 
Come up with a step by step plan (keep the plan brief - 3-4 words pointers is enough) to make a decision on whether to hold or sell the stock.
Show your planning before you execute any tools.
Answer only with the output from tools. Note that the tool selection is optional. However, be thorough with your analysis - use all the tools at your disposal to make an informed decision.

The sql tool has access to all the following information and can answer any question using the database:
{database_schema}. you can ask questions in natural language and the tool will convert them into SQL queries to fetch the required information.
when using the sql_tool make sure you are asking for the most recent data. send instruction to get data by ordering the table by data and picking the latest date from top unles you specifically need data from other dates.

You can use the different news tools to gather different news articles [relevant to company, and news about the industry keywords (figure out 2-3 industry keywords  based on the company profile)] and summarise them before coming to your conclusion
The next instruction is very impoertant: As you accrue data - keep summarizing the information within 10000 words to make a decision on whether to hold or sell the stock.
Remember to summarise the information you have gathereed at every step.Do not re-iterate your plan at every step.
You can choose to not use a tool and reply 'I am not able to determine whether to hold it or not. Here is the information I do have: ' along with summary of all the information you have gathered if the tools are not useful to answer the question.
or If answer is not available in the tool output - do the same and reply with the summary of all the information you have with a note.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

In [16]:
tools = [sql_tool,get_stock_price,insider_transactions,news_tool, catchall]

llm = AzureChatOpenAI(
    model_name=model_name,
    api_key=openai_key,
    azure_endpoint=azure_endpoint,
    api_version="2024-02-01",
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent.get_prompts()

[ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessa

## Testing Fin-Agent

In [17]:
query = "Should we sell or hold the johnson and johnson stock?"
response = agent_executor.invoke({"input": query})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `sql_tool` with `{'user_input': "SELECT * FROM assets_liabilities WHERE ticker = 'JNJ' ORDER BY date DESC LIMIT 1"}`
responded: ### Step-by-Step Plan:
1. **Financial Health**: Check assets, liabilities, and equity.
2. **Fundamentals**: Analyze key financial ratios and metrics.
3. **Stock Performance**: Review recent stock price and volume.
4. **Market Sentiment**: Gather recent news and insider transactions.

Let's start by gathering the financial health and fundamentals of Johnson & Johnson (JNJ).



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT ticker, date, total_assets, cash_on_hand, long_term_debt_excl_capitalized_leases, total_liabilities, total_share_holder_equity FROM assets_liabilities WHERE ticker = 'JNJ' ORDER BY date DESC LIMIT 1"}`


```sql
SELECT ticker, date, total_assets, cash_on_hand, long_term_debt_excl_capitalized_leases, total_liabilities, total_share_holder_equ

In [239]:
query = "Should we sell or hold the nvidia stock?"
response = agent_executor.invoke({"input": query})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `sql_tool` with `{'user_input': "SELECT * FROM assets_liabilities WHERE ticker = 'NVDA' ORDER BY date DESC LIMIT 1;"}`




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT "ticker", "date", "total_assets", "cash_on_hand", "long_term_debt_excl_capitalized_leases", "total_liabilities", "total_share_holder_equity" FROM "assets_liabilities" WHERE "ticker" = \'NVDA\' ORDER BY "date" DESC LIMIT 1;'}`


```sql
SELECT "ticker", "date", "total_assets", "cash_on_hand", "long_term_debt_excl_capitalized_leases", "total_liabilities", "total_share_holder_equity" 
FROM "assets_liabilities" 
WHERE "ticker" = 'NVDA' 
ORDER BY "date" DESC 
LIMIT 1;
```
Invoking: `sql_db_query` with `{'query': 'SELECT "ticker", "date", "total_assets", "cash_on_hand", "long_term_debt_excl_capitalized_leases", "total_liabilities", "total_share_holder_equity" FROM "assets_liabilities" WHERE "ticker" = \'NVDA\' ORDER BY "da

In [241]:
query = "Should we sell or hold the Charles Schwab Corporation stock?"
response = agent_executor.invoke({"input": query})
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `sql_tool` with `{'user_input': "SELECT * FROM assets_liabilities WHERE ticker = 'SCHW' ORDER BY date DESC LIMIT 1"}`




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT ticker, date, total_assets, cash_on_hand, long_term_debt_excl_capitalized_leases, total_liabilities, total_share_holder_equity FROM assets_liabilities WHERE ticker = 'SCHW' ORDER BY date DESC LIMIT 1"}`


```sql
SELECT ticker, date, total_assets, cash_on_hand, long_term_debt_excl_capitalized_leases, total_liabilities, total_share_holder_equity 
FROM assets_liabilities 
WHERE ticker = 'SCHW' 
ORDER BY date DESC 
LIMIT 1;
```
Invoking: `sql_db_query` with `{'query': "SELECT ticker, date, total_assets, cash_on_hand, long_term_debt_excl_capitalized_leases, total_liabilities, total_share_holder_equity FROM assets_liabilities WHERE ticker = 'SCHW' ORDER BY date DESC LIMIT 1"}`


[('SCHW', '2022-09-30', 577563.0, 90604.0, 2

Power Ahead!